# Connect-Four: Informed and Adversial Search Strategies

**PROBLEM FORMULATION**

    - Initial State: empty board (7x6); this is represented by a matrix full of 0's
    - Objective: to have a column, line or diagonal with 4 equal pieces next to each other (the pieces are represented by 1's and 2's, depending on the player)
    - Operators: place_piece(); place a piece in a given column

In [20]:
import time
import random
import heapq
import copy
# import pygame
from typing import List, Literal, Tuple, Callable
from typing_extensions import Self
# class Self: pass
from dataclasses import dataclass
from logging import error, info, basicConfig, INFO

basicConfig(
    level=INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

Board Implementation:

In [21]:
@dataclass
class Board:
    num_rows: int
    num_cols: int
    grid: List[int]
    curr_player: int
    score: float
    last_move : List[int]
    num_pieces: int

    def __init__(
        self,
        num_rows: int,
        num_cols: int,
        curr_player: int = 1,
        grid: List[int] = [],
        score: float = 0,
        last_move: List[int] = [-1, -1], 
    ) -> None:
        self.num_rows, self.num_cols = num_rows, num_cols
        self.curr_player = curr_player
        self.grid = grid or Board.initial_board(num_rows, num_cols)
        self.score = score
        self.last_move = last_move
        self.num_pieces = num_cols*num_rows - 4
        
        assert len(self.grid) == self.num_rows * self.num_cols
    
    def __eq__(self,other:Self):
        """
        checks if two boards are equal
        """
        if self.num_cols != other.num_cols or self.num_rows != other.num_rows or self.curr_player != other.curr_player:
            return False
        x = zip(self.grid,other.grid)
        for (i,j) in x:
            if i!=j:
                return False
        return True
    
    @staticmethod
    def initial_board(num_rows: int, num_cols: int) -> List[int]:
        return [0 for _ in range(num_cols * num_rows)] #initial state is the empty board
    
        """0.......0
           .........
           0.......0"""

    #to use in a dictionary (if needed)
    def __hash__(self):
        return hash((tuple(self.grid), self.curr_player))

    def is_valid_pos(self, x: int, y: int) -> bool:
        """
        Bounds check
        """
        return 0 <= x < self.num_cols and 0 <= y < self.num_rows
    
    def get_valid_actions(self) -> List[int]:
        """
        Get all actions for the current player
        """
        res = []
        i = 0
        for column in self.grid[0:self.num_cols]:
            #an action is valid if there's at least one empty slot in a given column (it's not full)
            if(column == 0):
                res.append(i)
            i += 1
                
        return res

    def get_piece(self, x: int, y: int) -> int:
        """
        Returns piece at x, y
        """
        #if piece isn't within boundaries, returns error
        if not self.is_valid_pos(x, y):
            raise IndexError(f"No such piece x:{x} y:{y}")
        return self.get_piece_unchecked(x, y)

    def get_piece_unchecked(self, x: int, y: int) -> int:
        """
        Returns piece at x, y without bounds check
        Use at your own risk
        """
        return self.grid[(y * self.num_cols) + x]

    def set_piece(self, x: int, y: int, val: int) -> None:
        """
        Sets piece at x, y to value x
        """
        if not self.is_valid_pos(x, y) or self.get_piece_unchecked(x, y) != 0:
            raise IndexError(f"No such piece x:{x} y:{y}")
        self.set_piece_unchecked(x, y, val)

    def set_piece_unchecked(self, x: int, y: int, val: int) -> None:
        """
        Sets piece at x, y to value x without bounds check
        Use at your own risk
        """
        self.grid[(y * self.num_cols) + x] = val

    def place_piece(self, x: int) -> Tuple[int ,int]:
        assert self.grid[x] == 0, f"Tried to place a piece in column {x} which is already full"

        y = self.fall_piece(x)
        self.set_piece(x, y, self.curr_player)
        self.curr_player = 3 - self.curr_player
        self.num_pieces -= 1

        self.last_move = [x, y]
        return (x, y)

    def is_terminal_piece(self, x: int, y: int) -> bool:
        if self.num_pieces == 0:
            return True
        
        # Diagonal \ (up -> bottom)
        count = 1
        for offset in range(-3, 4): #checks the 3 positions before, and the 3 positions after
            if offset == 0: continue

            cur_x = x + offset
            cur_y = y + offset
            if self.is_valid_pos(cur_x, cur_y) and self.get_piece_unchecked(cur_x, cur_y) == self.curr_player:
                count += 1
                if count >= 4: return True
            else:
                count = 1
        if count >= 4: return True

        # Diagonal / (bottom -> up)
        count = 1
        for offset in range(-3, 4):
            if offset == 0: continue

            cur_x = x + offset
            cur_y = y - offset
            if self.is_valid_pos(cur_x, cur_y) and self.get_piece_unchecked(cur_x, cur_y) == self.curr_player:
                count += 1
                if count >= 4: return True
            else:
                count = 1
        if count >= 4: return True

        # Horizontal -
        count = 1
        for offset in range(-3, 4):
            if offset == 0: continue

            cur_x = x + offset
            cur_y = y
            if self.is_valid_pos(cur_x, cur_y) and self.get_piece_unchecked(cur_x, cur_y) == self.curr_player:
                count += 1
                if count >= 4: return True
            else:
                count = 1
        if count >= 4: return True

        # Vertical |
        count = 1
        for offset in range(-3, 4):
            if offset == 0: continue

            cur_x = x
            cur_y = y + offset
            if self.is_valid_pos(cur_x, cur_y) and self.get_piece_unchecked(cur_x, cur_y) == self.curr_player:
                count += 1
                if count >= 4: return True
            else:
                count = 1
        
        return count >= 4
    

    def is_terminal_move(self, x: int) -> bool:
        y = self.fall_piece(x)
        return self.is_terminal_piece(x, y)

    def fall_piece(self, x: int) -> int:
        y = 0
        #given a row, the objective is finding the last row empty (piece is "falling")
        while y  < self.num_rows and self.get_piece(x, y) == 0:
            y += 1
        
        return y - 1

    def __str__(self) -> str:
        """
        write board
        """
        x = self.num_cols
        y = self.num_rows

        matrix = [[self.get_piece(i, j) for i in range(x)] for j in range(y)]

        ret = ""
        for row in matrix:
            ret += " ".join(map(str, row)) + "\n"

        ret += f"next player: {self.curr_player}"
        return ret


In [22]:
# Test terminal move
test_cases = [
    # Diagonal \
    ([
        0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
    ], (4, 2), False),
    ([
        0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
    ], (4, 2), True),
    ([
        0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0,
    ], (4, 2), False),
    ([
        0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0,
    ], (4, 2), False),
    ([
        0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0,
    ], (4, 2), True),
    ([
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0,
    ], (4, 2), True),

    # Diagonal /
    ([
        0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
    ], (2, 2), False),
    ([
        0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
    ], (2, 2), True),
    ([
        0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
    ], (2, 2), False),
    ([
        0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
    ], (2, 2), False),
    ([
        0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
    ], (2, 2), True),
    ([
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
    ], (2, 2), True),

    # Horizontal
    ([
        0, 0, 0, 0, 0, 0, 0,
        1, 1, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
    ], (2, 1), True),
    ([
        0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 1, 1, 1, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
    ], (2, 1), True),
    ([
        0, 0, 0, 0, 0, 0, 0,
        1, 1, 0, 0, 1, 1, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
    ], (3, 1), False),

    # Vertical
    ([
        0, 1, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
    ], (1, 2), True),
    ([
        0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
    ], (2, 2), False),
    ]

passed = True
for i, test in enumerate(test_cases):
    board = Board(6, 7, grid=test[0])
    if board.is_terminal_piece(test[1][0], test[1][1]) != test[2]:
        passed = False
        error(f"Test case {i} failed!\n" 
            + str(board) 
            + "\nx: " + str(test[1][0]) 
            + ", y: " + str(test[1][1]))
if passed: info("All tests passed")

2024-03-04 21:38:13 - INFO - All tests passed


In [23]:
# Test place piece
test_cases = [
    ([
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
    ],
    [
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0,
    ], 0),
    ([
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0,
    ],
    [
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0,
    ], 0),
    ]

passed = True
for i, test in enumerate(test_cases):
    board = Board(6, 7, grid=test[0])
    board.place_piece(test[2])
    if not all(x == y for x, y in zip(board.grid, test[1])):
        passed = False
        error(f"Test case {i} failed!\n" 
            + "Expected:\n" + str(test[1])
            + "\nGot:\n" + str(board.grid))

if passed: info("All tests passed")

2024-03-04 21:38:13 - INFO - All tests passed


In [24]:
class Renderer:
    pass

Implementing the game

In [25]:
@dataclass
class Game:
    """
    Holds the game settings
    """

    state: Board
    player1_AI: Callable[[Board], int]
    player2_AI: Callable[[Board], int]
    renderer: Renderer | None = None

    player1_AI_name: str = "Player 1"
    player2_AI_name: str = "Player 2"

    def start(self, log_mov=False) -> int:
        """
        Start a new game
        """
        self.state = Board(self.state.num_rows, self.state.num_cols)

        if self.renderer:
            self.renderer.render(self.state)
            #pygame.time.wait(500)

        result = -1
        while True:
            if self.state.curr_player == 1:
                move = self.player1_AI(self.state)
                print(f"Player 1 placed: {move}")
            else:
                move = self.player2_AI(self.state)
                print(f"Player 2 placed: {move}")


            result = self.state.is_terminal_move(move)
            self.state.place_piece(move)
            print(board.score)

            if self.renderer:
                self.renderer.render(self.state)

            if log_mov:
                print(self.state)
                
            if board.num_pieces == 0:
                result = 0
                break
            
            if result:
                result =  3-board.curr_player
                break


            if self.renderer:
                pygame.time.wait(500)

        if self.renderer:
            if result == 3:
                self.renderer.show_title("Nobody wins")
            else:
                self.renderer.show_title(f"Player {result} Wins")
            #pygame.display.flip()
            #pygame.time.wait(2000)

        return result

    def run_n_matches(self, n: int, max_time: int = 3600, log_moves: bool = False):
        """
        utility function to automate n matches execution
        returns the total distribution of players wins and draws
        """
        start_time = time.time()

        results = [0, 0, 0]  # [player 1 victories, player 2 victories, draws]

        turns = []
        remaining = n
        while remaining > 0 and time.time() - start_time < max_time:
            remaining -= 1
            result = self.start(log_moves)
            results[result - 1] += 1
            turns.append(self.state.num_pieces)

        # Statistics
        elapsed = int(time.time() - start_time)
        turns_avg = sum(turns) / (n - remaining)

        print("\n=== Elapsed time: %s seconds ===" % (elapsed))
        print(f"  Matches played: {n-remaining}")
        print()
        print(f"  {self.player1_AI_name}: {results[0]} victories")
        print(f"  {self.player2_AI_name}: {results[1]} victories")
        print(f"  Draws: {results[2]} ")
        print(f"  Win ratio (player 1): {results[0]/(n-remaining):.2f}")
        print(f"  Win ratio (player 2): {results[1]/(n-remaining):.2f}")
        print()
        print(f"  Turns MIN: {min(turns)}")
        print(f"  Turns MAX: {max(turns)}")
        print(f"  Turns AVG: {turns_avg:.2f}")
        print()
        print(f"  AVG time per game: {elapsed/(n-remaining):.2f} s")
        print(f"  AVG time per turn: {elapsed/sum(turns):.2f} s")

        print("===============================")
        # --------------------------------------------------#

# A* Algorithm

In [26]:
@dataclass
class Node:
    board: Board
    next_move: int
    g: int
    f: int
    ancestor: int

    def __lt__(n1: Self, n2: Self):
        # A*, one node is 'better' than the other when it has a lower f cost.
        # tiebreak on smaller g
        return (n1.f < n2.f) or (n1.f == n2.f and n1.g < n2.g)
    

In [27]:
@dataclass
class Stats:
    nodes_expanded: int = 0
    to_visit_size: int = 0

stats = Stats()

In [28]:
def execute_a_star_move(
    h: Callable[[Board], float], depth: int
) -> Callable[[Board], int]:
    def execute_a_star_move_aux(start_board: Board) -> int:
        # TODO: max depth, terminal only if the correct player wins
        player = start_board.curr_player
        to_visit: List[Node] = []
        for move in start_board.get_valid_actions():
            board = copy.deepcopy(start_board)
            board.place_piece(move)

            node = Node(start_board, move, 0, -h(board), move)
            heapq.heappush(to_visit, node)

        visited = set()

        while len(to_visit) != 0:
            # Pop the best node off the to_visit list (+ goal check)
            expand_me = to_visit[0]
            if expand_me.board.is_terminal_move(expand_me.next_move):
                if expand_me.board.curr_player == player:
                    # We reached the goal
                    break
                else:
                    # The enemy wins D-:
                    heapq.heappop(to_visit)
                    visited.add((expand_me.board, expand_me.next_move))
                    continue

            visited.add((expand_me.board, expand_me.next_move))
            new_board = copy.deepcopy(expand_me.board)
            new_board.place_piece(expand_me.next_move)
            print(new_board.score)
            print(new_board)

            stats.nodes_expanded += 1
            moves = new_board.get_valid_actions()
            heapq.heappop(to_visit)

            # Add each neighbour
            for move in moves:
                if (new_board, move) in visited:
                    continue

                cost = 10 # TODO: customizable?
                g = expand_me.g + cost

                idx = index_of(to_visit, new_board, move)
                if idx == -1:
                    # print(new_board)
                    # print(move)
                    # print(h(new_board))
                    node = Node(new_board, move, g, g - h(new_board), expand_me.ancestor)
                    heapq.heappush(to_visit, node)
                else:
                    node = to_visit[idx]
                    if g < node.g:
                        node.f += g - node.g
                        node.g = g
                        node.ancestor = expand_me.ancestor
                        heap_repair(to_visit, idx)

        stats.to_visit_size += len(to_visit)
        assert expand_me, f"Board has no valid actions {expand_me.board}"
        return expand_me.ancestor

    return execute_a_star_move_aux

def index_of(nodes: List[Node], board: Board, move: int) -> int:
    for i, x in enumerate(nodes):
        if x.next_move == move and x.board == board:
            return i
    
    return -1

def heap_repair(list: List[Node], ii: int):
    while True:
        parent = (ii + 1) // 2 - 1
        if parent < 0:
            break
        if not list[ii] < list[parent]:
            break
        list[ii], list[parent] = list[parent], list[i]
        ii = parent

In [29]:
def heuristic(grid: Board) -> float:
    
    x, y = grid.last_move
    player = 3 - grid.curr_player
    
    finalcount = 0 #peca sozinha somada no fim se um dos quatro cantos falhar
    strikes = 0
    scores = [0, 10, 50, 512, 512, 512, 512]
    
    #Vertical | (não existem em cima)
    count = 0
    for offset in range(1, 4):
        cur_x = x
        cur_y = y + offset
        
        if grid.is_valid_pos(cur_x, cur_y) and grid.get_piece_unchecked(cur_x, cur_y) == player:
            count+=1
        else:
            break
            
    #print(f"vertical - {count} \n count-{scores[count]}")
            
    if count!= 0:
        finalcount += scores[count] - scores[count-1]
    else:
        strikes += 1
        
    #Horizontal _ 
    count = 0
    for offset in range(-3, 4):
        cur_x = x + offset
        cur_y = y
        
        if grid.is_valid_pos(cur_x, cur_y) and grid.get_piece_unchecked(cur_x, cur_y) == player:
            count +=1
        else:
            if cur_x > x:
                break
            count = 0
     
    #print(f"horizontal - {count}\n count-{scores[count-1]}")
    if count != 1:
        finalcount += scores[count-1] - scores[count-2]
    else:
        strikes += 1
        
    #Diagonal / (bottom -> up)
    count = 0
    for offset in range(-3, 4):
        cur_x = x + offset
        cur_y = y - offset
        
        if grid.is_valid_pos(cur_x, cur_y) and grid.get_piece_unchecked(cur_x, cur_y) == player:
            count +=1
        else:
            if cur_x > x:
                break
            count = 0
    
    #print(f"diag1 - {count}\n count-{scores[count-1]}")
    if count != 1:
        finalcount += scores[count-1] - scores[count-2]
    else:
        strikes += 1
    
    
    
    #Diagonal \ (up -> bottom)
    count = 0
    for offset in range(-3, 4):
        cur_x = x + offset
        cur_y = y + offset
        
        if grid.is_valid_pos(cur_x, cur_y) and grid.get_piece_unchecked(cur_x, cur_y) == player:
            count +=1
        else:
            if cur_x > x:
                break
            count = 0
    
    #print(f"diag2 - {count}\n count-{scores[count-1]}")
    if count != 1:
        finalcount += scores[count-1] - scores[count-2]
    else:
        strikes += 1
        
    if strikes == 4:
        finalcount += 1
    
    if finalcount > 0:
        finalcount += 16
    else:
        finalcount -= 16
    
    if player == 2:
        finalcount *= -1
    #print(f"final count sm - {finalcount}")
    grid.score += finalcount
    
    return grid.score

In [30]:
def heuristic2(grid: Board) -> float:

    player = 3 - grid.curr_player

    scores = [1,10,50,75,512]
    
    final_count = 0
    visited_vertically = []
    visited_horizontally = []
    visited_diag_rl = []
    visited_diag_lr = []

    #TODO: vertical,horizontal,diagonals,block,win/loss/draw
    for row in grid.num_rows:
        for col in grid.num_cols:
            #check vertically (only goes down)
            for offset in range (1,4):
                cur_row = row + offset
                if grid.is_valid_pos(cur_row, col) and grid.get_piece_unchecked(cur_row, col) == player and (cur_row,col) not in visited_vertically:
                    count +=1
                    visited_vertically.append(cur_row,col)
                else:
                    break
            if count >= 4: final_count += scores[-1]
            if grid.get_piece_unchecked(row+1,col) == grid.curr_player: #blocking oponent
                final_count += scores[-2]
            else: final_count += scores[count]

            #check horizontally
            for offset in range(-3,4):
                cur_col = col + offset
                if grid.is_valid_pos(row, cur_col) and grid.get_piece_unchecked(row, cur_col) == player and (row,cur_col) not in visited_horizontally:
                    count +=1
                    visited_horizontally.append(row,cur_col)
                else:
                    break
            if count >= 4: final_count += scores[-1]
            if grid.get_piece_unchecked(row,col-1) == grid.curr_player or grid.get_piece_unchecked(row,col+1) == grid.curr_player:
                final_count += scores[-2]
            else: final_count += scores[count]

            #check diagonally (right to left)
            for offset in range(-3,4):
                cur_row = row + offset
                cur_col = col - offset
                if grid.is_valid_pos(cur_row, cur_col) and grid.get_piece_unchecked(cur_row, cur_col) == player and (cur_row,cur_col) not in visited_diag_rl:
                    count +=1
                    visited_horizontally.append(cur_row,cur_col)
                else:
                    break
            if count >= 4: final_count += scores[-1]
            if grid.get_piece_unchecked(row+1,col-1) == grid.curr_player or grid.get_piece_unchecked(row-1,col+1) == grid.curr_player:
                final_count += scores[-2]
            else: final_count += scores[count]

            #check diagonally (left to right)
            for offset in range(-3,4):
                cur_row = row - offset
                cur_col = col + offset
                if grid.is_valid_pos(cur_row, cur_col) and grid.get_piece_unchecked(cur_row, cur_col) == player and (cur_row,cur_col) not in visited_diag_lr:
                    count +=1
                    visited_horizontally.append(cur_row,cur_col)
                else:
                    break
            if count >= 4: final_count += scores[-1]
            if grid.get_piece_unchecked(row-1,col+1) == grid.curr_player or grid.get_piece_unchecked(row+1,col-1) == grid.curr_player:
                final_count += scores[-2]
            else: final_count += scores[count]
                
    return final_count

In [31]:
test_cases = [
    ([
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 2, 0, 0, 0,
        2, 0, 2, 2, 2, 0, 0,
    ], -31, [2, 5], -97),

    ([
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        2, 0, 2, 2, 2, 0, 0,
    ], -11, [2, 5], -67),

    ([
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        2, 0, 0, 2, 0, 0, 0,
        2, 0, 2, 2, 0, 0, 0,
    ], -20, [2, 5], -56),
]

passed = True
for i, test in enumerate(test_cases):
    board = Board(6, 7, 1, test[0], test[1], test[2])
    if (score := heuristic(board)) != test[3]:
        passed = False
        error(f"Test case {i} failed!\n" 
            + "Expected:\n" + str(test[3])
            + "\nGot:\n" + str(score))

if passed: info("All tests passed")


2024-03-04 21:38:13 - INFO - All tests passed


In [32]:
grid = [
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        2, 0, 2, 2, 0, 0, 0,
    ]
board = Board(6, 7, 1, grid, 11)
print(board)


stats = Stats()
a_star = execute_a_star_move(heuristic, 0)
move = a_star(board)

print()
print("Decision:", move)
board.place_piece(move)
print(board)

print()
print(stats)

0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
2 0 2 2 0 0 0
next player: 1
11
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
1 0 0 0 0 0 0
2 0 2 2 0 0 0
next player: 2
130
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
1 0 0 0 0 0 0
2 0 2 2 0 0 2
next player: 1
130
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
1 0 0 0 0 0 0
2 0 2 2 0 2 0
next player: 1
11
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
1 0 0 0 0 0 0
1 0 0 0 0 0 0
2 0 2 2 0 0 2
next player: 2
193
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
1 0 0 0 0 0 0
1 0 0 0 0 0 2
2 0 2 2 0 0 2
next player: 1
193
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
1 0 0 0 0 0 0
1 0 0 0 0 0 0
2 0 2 2 0 2 2
next player: 1
11
0 0 0 0 0 0 0
0 0 0 0 0 0 0
1 0 0 0 0 0 0
1 0 0 0 0 0 0
1 0 0 0 0 0 2
2 0 2 2 0 0 2
next player: 2
403
0 0 0 0 0 0 0
0 0 0 0 0 0 0
1 0 0 0 0 0 0
1 0 0 0 0 0 2
1 0 0 0 0 0 2
2 0 2 2 0 0 2
next player: 1
403
0 0 0 0 0 0 0
0 0 0 0 0 0 0
1 0 0 0 0 0 0
1 0 0 0 0 0 0
1 0 0 0 0 0 2
2 0 2 

In [33]:
def human_move(board: Board) -> int:
    return int(input("Jogada: "))


In [34]:
def random_move(board: Board) -> int:
    choise = random.choice(board.get_valid_actions())
    bak = copy.deepcopy(board)
    bak.place_piece(choise)
    board.score = bak.score
    heuristic(board)
    return choise

In [35]:
stats

Stats(nodes_expanded=9, to_visit_size=61)

In [36]:
test_game = Game(Board(6,7), random_move, execute_a_star_move(heuristic, -1))
test_game.run_n_matches(10, log_moves=True)

Player 1 placed: 4
11
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 1 0 0
next player: 2
16
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
2 0 0 0 1 0 0
next player: 1
16
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 2 0 1 0 0
next player: 1
16
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 1 2 0
next player: 1
16
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 1 0 2
next player: 1
16
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 2 0 0 1 0 0
next player: 1
16
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 2 0 0
0 0 0 0 1 0 0
next player: 1
16
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 2 1 0 0
next player: 1
-103
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
1 0 0 0 0 0 0
2 0 0 0 1 0 0
next player: 2
16
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
1 

In [ ]:
board = Board(6, 7)
while True:
    move = int(input())
    board.place_piece(move)
    print(board)
    print("Score:", heuristic(board))

0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
1 0 0 0 0 0 0
next player: 2
Score: 17
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
1 2 0 0 0 0 0
next player: 1
Score: 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
1 0 0 0 0 0 0
1 2 0 0 0 0 0
next player: 2
Score: 26
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
2 0 0 0 0 0 0
1 0 0 0 0 0 0
1 2 0 0 0 0 0
next player: 1
Score: 9
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
2 0 0 0 0 0 0
1 0 0 0 0 0 0
1 2 1 0 0 0 0
next player: 2
Score: 26
0 0 0 0 0 0 0
0 0 0 0 0 0 0
2 0 0 0 0 0 0
2 0 0 0 0 0 0
1 0 0 0 0 0 0
1 2 1 0 0 0 0
next player: 1
Score: 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
2 0 0 0 0 0 0
2 0 0 0 0 0 0
1 0 0 0 0 0 0
1 2 1 1 0 0 0
next player: 2
Score: 26
0 0 0 0 0 0 0
2 0 0 0 0 0 0
2 0 0 0 0 0 0
2 0 0 0 0 0 0
1 0 0 0 0 0 0
1 2 1 1 0 0 0
next player: 1
Score: -30
1 0 0 0 0 0 0
2 0 0 0 0 0 0
2 0 0 0 0 0 0
2 0 0 0 0 0 0
1 0 0 0 0 0 0
1 2 1 1 0 0 0
next player: 2
Score: -13


AssertionError: Tried to place a piece in column 7 which is already full

In [ ]:
def execute_mcts_move(
    h: Callable<[[Board], float], num_rounds: int
) -> Callable[[Board], bool]:
    def execute_mcts_move_aux(start_board: Board) -> int:
        for i in range(num_rounds):
            # Select
            selected_node = select(board)

            if not self.mdp.is_terminal(selected_node):

                child = selected_node.expand()
                reward = self.simulate(child)
                selected_node.back_propagate(reward, child)



    return execute_mcts_move_aux

def select(board: Board) -> Node:
    

SyntaxError: incomplete input (2952520562.py, line 20)

In [ ]:
def main_menu_phase(renderer: Renderer | None):
    """
    Main menu loop
    """
    if renderer:
        renderer.intro_screen()


def gameplay_phase(num_rows, num_cols, renderer: Renderer | None = None):
    """
    Gameplay loop
    """
    s = State(Board(num_rows, num_cols))
    escolh_game = {
        1: lambda x, y: execute_random_move,
        2: execute_minimax_move,
        3: lambda x, y: execute_player_move,
        4: execute_negamax_move,
        5: execute_minimax_move_with_transposition,
    }

    game = Game(
        s,
        escolh_game[jogo[0][0]](eval_1, jogo[0][1]),
        escolh_game[jogo[1][0]](eval_1, jogo[1][1]),
        renderer,
    )

    game.start()

In [ ]:

num_rows = 5
num_cols = 5
# renderer = Renderer(num_rows, num_cols)
renderer = None

while True:
    main_menu_phase(renderer)
    gameplay_phase(num_rows, num_cols, renderer)